In [1]:
import pandas as pd 
import numpy as np 
import neo4j

import seaborn as sns
import warnings
from neo4j import GraphDatabase
warnings.filterwarnings('ignore')
import networkx as nx
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from collections import Counter
##from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'neo4j'

In [ ]:
url = "neo4j+s://9d0eb28d.databases.neo4j.io"
username = "neo4j"
password = ""

driver = GraphDatabase.driver(url, auth=(username, password))
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n)")
    count = result.single()[0]
    print(f"La conexión con Aura Neo4j se ha establecido correctamente. Hay {count} nodos en la base de datos.")

In [ ]:
# Definir las listas de usuarios y películas permitidos
import random
usuarios = [1, 2, 666, 7]
peliculas = ['tt0000009', 'tt0000147', 'tt0000502', 'tt0000574', 'tt0000591', 'tt0000615', 'tt0000630', 'tt0000675', 'tt0000679', 
             'tt0000838', 'tt0000842', 'tt0000846', 'tt0000850', 'tt0000859', 'tt0000862', 'tt0000867', 'tt0000868', 'tt0000869', 
             'tt0000879', 'tt0000886', 'tt0000941', 'tt0000947', 'tt0000959', 'tt0000984', 'tt0001007', 'tt0001010', 'tt0001028', 
             'tt0001043', 'tt0001049', 'tt0001051', 'tt0001059', 'tt0001069', 'tt0001080', 'tt0001081', 'tt0001082', 'tt0001101', 
             'tt0001038','tt0001112', 'tt0001113', 'tt0001115', 'tt0001116', 'tt0001122', 'tt0001141', 'tt0001142', 'tt0001147',  
             'tt0001175', 'tt0001179','tt0001159', 'tt0001163','tt0001109']

# Generar una lista aleatoria de usuarios y películas
usuarios_aleatorios = random.choices(usuarios, k=100)
peliculas_aleatorias = random.choices(peliculas, k=100)

# Crear el dataframe con las columnas "usuario" y "pelicula"
visto = pd.DataFrame({'usuario_id': usuarios_aleatorios, 'pelicula_id': peliculas_aleatorias})



In [ ]:
for index, row in visto.iterrows():
    user_id = row["usuario_id"]
    movie_id = row['pelicula_id']      
    with driver.session() as session:    
        session.execute_write(
            lambda tx: tx.run("MATCH(u:User {id: $u_id})"
                              "MATCH (m:Movie {id: $id}) "
                              "CREATE (u)-[:IN_VISTAS]->(m)"
                               ,id=movie_id, u_id = user_id))

In [2]:
#recomendaciones (pagerank)
from gettext import find


query = """MATCH MATCH (G:Genre {name: "Action"})-[IN_GENRE]->(m:Movie)
RETURN G.name, m.name
"""
def películas_cliente(tx, password):
    result = tx.run(query, id=password)
    return list(result)                 #lista de peliculas
def pelis_action(password):
    records=[]
    with driver.session() as session:
        result=session.read_transaction(películas_cliente, password)
        for record in result:
            records.append(record)
    print("Found {record} records!". format(record=len(records)))
    return records
a= pelis_action(1)   #Resultados películas género acción

NameError: name 'driver' is not defined

In [ ]:
def create_networkx_graph(password):
    
    val_count  = Counter([(res['m.nombre'],res['m2.nombre']) for res in pelis_action(password) ])
    if len(val_count)==0:
        print(f" no records for {password}")
    print(f"  no of  records for {password} {len(val_count)}")

    recs = pd.DataFrame([[a[0], a[1], val_count[a]] for a in val_count])

    G = nx.DiGraph()
    #recs= recs.head(20)
    total = len(val_count)
    for idx , rec in recs.iterrows():
        #print(f"{idx} of {total} loaded")
        G.add_edge(rec[0], rec[1], weight = rec[2])
    return G
G= create_networkx_graph(password =1)

In [ ]:
def run_page_rank(G, password):
    pr=pagerank(G)
    sorted_pr = {k: v for k, v in sorted(pagerank(G).items(), key=lambda item: item[1], reverse = True)}
    df= pd.DataFrame(sorted_pr.items()).assign(password= password)
    df['rank'] = df[1].rank(method='dense', ascending=False)
    print(f"done calculating page rank")
    return df
df = run_page_rank(G,1)

In [ ]:
print(df.head(30))